Mann Whitney to test kpca distribution difference mild vs severe

In [2]:
import pickle
import tensorly as tl
import numpy as np
import pandas as pd
import tlviz
from sklearn.preprocessing import scale
from tensorly.decomposition import parafac2
from tlviz.utils import normalise
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy.linalg as la
import sspa
import scipy.stats as stats
from pathway_feature_selection import bootstrap_feature_select_pr2, feature_select_bs_ci, significant_pathways, harmonic_pvalue,combined_pseudoz, significant_pseudoz, plot_zscore_distribution

In [39]:
with open('metab_kpca.pkl','rb') as f:
    metab_kpca = pickle.load(f)

with open('prot_kpca.pkl','rb') as f:
    prot_kpca = pickle.load(f)

with open('metadata_binary.pkl','rb') as f:
    metadata_binary = pickle.load(f)
    
with open('p1_sigpath_dict.pkl','rb') as f:
    p1_sigpath_dict = pickle.load(f) 

with open('parafac2_common_metab_sigpath_dict.pkl','rb') as f:
    parafac2_common_metab_sigpath_dict = pickle.load(f)

with open('parafac2_common_prot_sigpath_dict.pkl','rb') as f:
    parafac2_common_prot_sigpath_dict = pickle.load(f)

with open('parafac2_all_prot_sigpath_dict.pkl','rb') as f:
    parafac2_all_prot_sigpath_dict = pickle.load(f)

with open('parafac2_all_metab_sigpath_dict.pkl','rb') as f:
    parafac2_all_metab_sigpath_dict = pickle.load(f)

In [45]:
parafac1_sig_path_set = set(p1_sigpath_dict.keys())
parafac2_common_metab_sigpath_set = set(parafac2_common_metab_sigpath_dict.keys())
parafac2_common_prot_sigpath_set = set(parafac2_common_prot_sigpath_dict.keys())
p2_common = parafac2_common_metab_sigpath_set.union(parafac2_common_prot_sigpath_set)
parafac2_all_prot_sigpath_set = set(parafac2_all_prot_sigpath_dict.keys())
parafac2_all_metab_sigpath_set = set(parafac2_all_metab_sigpath_dict.keys())
p2_all = parafac2_all_prot_sigpath_set.union(parafac2_all_metab_sigpath_set)

In [46]:
all_sig = parafac1_sig_path_set | p2_common | p2_all

In [49]:
all_sig = list(all_sig)

In [4]:
mild_id = []
severe_id = []
for index, cond in enumerate(metadata_binary):
    for index, id in enumerate(metab_kpca.index):
        if cond == 0:
            mild_id.append(id)
    else:
        severe_id.append(id)

In [5]:
# metab

metab_ps = []
for column in metab_kpca.columns:
    mild_group = metab_kpca.loc[mild_id][column]
    severe_group = metab_kpca.loc[severe_id][column]
    _, p_value = stats.mannwhitneyu(mild_group, severe_group, alternative='two-sided')
    metab_ps.append(p_value)
df_metab_ps=pd.DataFrame({'p_val':metab_ps},index=metab_kpca.columns)


In [6]:
df_metab_ps.sort_values(by='p_val',ascending=False)

,p_val
R-HSA-9006934,5.431834e-01
R-HSA-15869,1.284971e-01
R-HSA-445717,6.814124e-02
R-HSA-8956319,6.814124e-02
R-HSA-8956321,3.332979e-02
...,...
R-HSA-189445,2.995133e-79
R-HSA-425366,2.995133e-79
R-HSA-71387,2.995133e-79
R-HSA-442660,9.135948e-82


In [58]:
from statsmodels.stats.multitest import multipletests
p_values = df_metab_ps['p_val'].to_list()
alpha = 0.05
corrected_results = multipletests(p_values, alpha=alpha, method='bonferroni')
print(f"Metabolomics Bonferroni corrected alpha is {corrected_results[3]}")

Metabolomics Bonferroni corrected alpha is 0.00033557046979865775


In [59]:
df_metab_ps['Reject'] = corrected_results[0]
metab_count_false_rejs = df_metab_ps.loc[df_metab_ps.index.isin(all_sig) & (df_metab_ps['Reject']== False)].shape[0]
metab_count_false_rejs

8

In [63]:
metab_false_rejs = df_metab_ps.loc[df_metab_ps.index.isin(all_sig) & (df_metab_ps['Reject']== False)]
metab_false_rejs

,p_val,Reject
R-HSA-15869,0.128497,False
R-HSA-194138,0.033330,False
R-HSA-4420097,0.033330,False
R-HSA-549127,0.015011,False
R-HSA-8939211,0.015011,False
R-HSA-9006931,0.015011,False
R-HSA-9006934,0.543183,False
R-HSA-9009391,0.015011,False


In [8]:
prot_ps = []
for column in prot_kpca.columns:
    mild_group = prot_kpca.loc[mild_id][column]
    severe_group = prot_kpca.loc[severe_id][column]
    _, p_value = stats.mannwhitneyu(mild_group, severe_group, alternative='two-sided')
    prot_ps.append(p_value)
df_prot_ps=pd.DataFrame({'p_val':prot_ps},index=prot_kpca.columns)

In [9]:
df_prot_ps.sort_values(by='p_val',ascending=False)

,p_val
R-HSA-382551,1.000000e+00
R-HSA-3000480,7.611357e-01
R-HSA-5218921,5.431834e-01
R-HSA-190239,5.431834e-01
R-HSA-190372,5.431834e-01
...,...
R-HSA-8986944,2.995133e-79
R-HSA-1251985,9.135948e-82
R-HSA-5684996,9.135948e-82
R-HSA-1236394,9.135948e-82


In [ ]:
p_values = df_prot_ps['p_val'].to_list()
alpha = 0.05
corrected_results = multipletests(p_values, alpha=alpha, method='bonferroni')

In [53]:
corrected_results[3]

0.00012953367875647668

In [42]:
df_prot_ps['Reject'] = corrected_results[0]

In [55]:
prot_count_false_rejs = df_prot_ps.loc[df_prot_ps.index.isin(all_sig) & (df_prot_ps['Reject']== False)].shape[0]
prot_count_false_rejs

8

In [74]:
prot_false_rejs = df_prot_ps.loc[df_prot_ps.index.isin(all_sig) & (df_prot_ps['Reject']== False)]
prot_false_rejs

,p_val,Reject
R-HSA-5693532,0.223970,False
R-HSA-5693538,0.223970,False
R-HSA-168255,0.223970,False
R-HSA-15869,0.000825,False
R-HSA-389948,0.000825,False
R-HSA-9679191,0.015011,False
R-HSA-381340,0.015011,False
R-HSA-4420097,0.002365,False


In [66]:
metab_false_rejs

,p_val,Reject
R-HSA-15869,0.128497,False
R-HSA-194138,0.033330,False
R-HSA-4420097,0.033330,False
R-HSA-549127,0.015011,False
R-HSA-8939211,0.015011,False
R-HSA-9006931,0.015011,False
R-HSA-9006934,0.543183,False
R-HSA-9009391,0.015011,False


In [78]:
prot_false_rejs

,p_val,Reject
R-HSA-5693532,0.223970,False
R-HSA-5693538,0.223970,False
R-HSA-168255,0.223970,False
R-HSA-15869,0.000825,False
R-HSA-389948,0.000825,False
R-HSA-9679191,0.015011,False
R-HSA-381340,0.015011,False
R-HSA-4420097,0.002365,False


In [80]:
(set(prot_false_rejs.index)).intersection(metab_false_rejs.index)

{'R-HSA-15869', 'R-HSA-4420097'}

In [83]:
false_paths = list((set(prot_false_rejs.index)).union(metab_false_rejs.index))

In [85]:
len(false_paths)

14

In [88]:
false_in_p1_common = []
false_in_p2_common = []
false_in_p2_all = []
for path in false_paths:
    if path in p1_sigpath_dict.keys():
        false_in_p1_common.append(path)
    elif path in list(p2_common):
        false_in_p2_common.append(path)
    else:
        false_in_p2_all.append(path)
print(len(false_in_p1_common),len(false_in_p2_common),len(false_in_p2_all))

1 4 9


In [89]:
false_in_p1_common

['R-HSA-381340']

In [25]:
metabp = float(df_prot_ps.loc['R-HSA-1643685'])

/var/folders/7h/31w4d6n52ds2j9y7fxh5ngjc0000gn/T/ipykernel_57925/2503461512.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  metabp = float(df_prot_ps.loc['R-HSA-1643685'])


In [27]:
protp = float(df_metab_ps.loc['R-HSA-1643685'])

/var/folders/7h/31w4d6n52ds2j9y7fxh5ngjc0000gn/T/ipykernel_57925/2443212946.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  protp = float(df_metab_ps.loc['R-HSA-1643685'])


In [29]:
hmp = 2*metabp*protp/(metabp+protp)
hmp

6.22781054375223e-48

In [31]:
metabp = float(df_prot_ps.loc['R-HSA-168256'])
protp = float(df_metab_ps.loc['R-HSA-168256'])
hmp = 2*metabp*protp/(metabp+protp)
hmp

/var/folders/7h/31w4d6n52ds2j9y7fxh5ngjc0000gn/T/ipykernel_57925/1768049086.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  metabp = float(df_prot_ps.loc['R-HSA-168256'])
/var/folders/7h/31w4d6n52ds2j9y7fxh5ngjc0000gn/T/ipykernel_57925/1768049086.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  protp = float(df_metab_ps.loc['R-HSA-168256'])


5.144197440787791e-46

In [10]:
with open('mw_metab.pkl', 'wb') as f:
    pickle.dump(df_metab_ps, f)
with open('mw_prot.pkl', 'wb') as f:
    pickle.dump(df_prot_ps, f)